ipypixano exposes the 2D elements of Pixano Elements in a jupyter notebook. 

Pixano Elements is a library of web components dedicated to data annotation task. 

https://github.com/pixano/pixano-elements

Currently the following items are available:

* pxn-rectangle
* pxn-polygon
* pxn-segmentation
* pxn-smart-rectangle
* pxn-smart-segmentation


To use ipypixano , just install it with pip




```
pip install ipypixano
```

This notebook presents how to create elements, configure them and how to retrieve annotation data in the python side.


In [2]:
from  ipypixano import Pixano


For now the package has some limitations and undesirable behaviors:

Keyboard shortcuts used in pixano elements are always active even outside the cell displaying the element. For example,  the 'm', 'p' and 'h' keys will respectively darken, lighten and hide/show the annotations in the element. 

Similarly ctr-C and ctr-V will copy the annotations or the segmentation mask when they are used (in this case use the copy/paste from the context menu displayed by the right mouse button).

Moreover, if the same element is displayed twice, the annotations are lost when the second element is displayed.


### Element creation : 

* 'element' indicates the type of element to create (see list above)
* 'image' indicates the image to display, image can be a filename (regardless of its location on your disk), a url, a numpy array or an Image loaded by PIL

In [3]:
w_rect = Pixano(element='pxn-smart-rectangle',image='image.jpg')
w_rect

Pixano(value=None, element='pxn-smart-rectangle', image='/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHBwgHBgoICAgLCgoLD…

In [32]:
w_rect.shapes

[{'id': 'kk2hr',
  'color': 'red',
  'geometry': {'vertices': [0.23828125,
    0.16944444444444445,
    0.340625,
    0.35833333333333334],
   'type': 'rectangle'},
  'category': 'class4',
  'options': {}},
 {'id': '9ubx9d',
  'color': '#eca0a0',
  'geometry': {'vertices': [0.44140625,
    0.13194444444444445,
    0.54296875,
    0.2791666666666667],
   'type': 'rectangle'},
  'category': 'class2',
  'options': {}},
 {'id': '19ezwk',
  'color': 'blue',
  'geometry': {'vertices': [0.49453125,
    0.5055555555555555,
    0.60703125,
    0.6958333333333333],
   'type': 'rectangle'},
  'category': 'class1',
  'options': {}},
 {'id': 'rg38rh',
  'color': 'green',
  'geometry': {'vertices': [0.1484375,
    0.5222222222222223,
    0.23515625,
    0.6458333333333334],
   'type': 'rectangle'},
  'category': 'class3',
  'options': {'checkbox example': False,
   'dropdown example': 'something',
   'textfield example': 'some text'}},
 {'id': 's0fqin',
  'color': 'green',
  'geometry': {'vertices':

In [ ]:
w_poly = Pixano(element='pxn-polygon',image='https://raw.githubusercontent.com/pixano/pixano-elements/master/demos/rectangle/image.jpg')
w_poly

In [21]:
import cv2
img=cv2.imread("traffic.jpg")
w_seg = Pixano(element='pxn-segmentation',image=img)
w_seg

/home/nallezard/anaconda3/lib/python3.6/site-packages/traitlets/traitlets.py:567: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


Pixano(value=None, element='pxn-segmentation', image='iVBORw0KGgoAAAANSUhEUgAABkAAAAH0CAIAAADqknYdAAAgAElEQVR4…

###  Annotations retrieval




#### Rectangle et Polygon elements

Annotations created in *-rectangle or *-polygon elements can be retrieved by the ```shapes``` attribute.

```shapes``` is a list of dictionaries describing each rectangle or polygon.

In [ ]:
w_rect.shapes

Annotation data can be modified from python using the ```setShapes(annot)``` method where annot is a list of annotations with the same structure as that returned by shapes

In [ ]:
new_annot=[{'id': 'toto',
  'color': 'blue',
  'geometry': {'vertices': [0.2,0.2,0.5,0.5],'type': 'rectangle'}}]

w_rect.setShapes(new_annot)

You can choose the category of new annotations created using the attribute ```current_category``` 

In [ ]:
w_rect.current_category='dog'

The color of categories can be changed using the attribute ```categories_colors```

In [ ]:
w_rect.categories_colors={'dog':'green','cat':'gray','car':'yellow'}

Use ```clearShapes()``` to delete annotations

In [ ]:
w_rect.clearShapes()

Selected annotations can be found in the ```selectedShapeIds``` attribute which returns a list of ids

In [ ]:
w_rect.selectedShapeIds

#### Integration example

The following example shows how to observe changes on ```selectedShapeIds```  and call a function displaying informations of selected items in an ipywidgets Label

First create somes annotations in the pixano element, then select them to see changes displayed below the element.

In [ ]:
from ipywidgets import Label,VBox
selected=Label(value="shape selected")
w_rect = Pixano(element='pxn-rectangle',image='image.jpg')

    
def on_change_selected(change):
    if len(change['new'])>0:
        idseleted=change['new'][0]
        shapes=w_rect.shapes
        for s in shapes:
            if s['id']==idseleted:
                vertices=s["geometry"]["vertices"] 
                vertices_str=",".join(list(map(lambda x:"{:.2f}".format(x),vertices)))
                selected.value="Shape selected : "+change['new'][0]+ "\nvertices :\n"+vertices_str      
    else:
        selected.value="Shape selected : None"
    
w_rect.observe(on_change_selected, names='selectedShapeIds')

VBox([w_rect,selected])

### Segmentation Element


Segmentation annotations are returned as a mask in ndarray format using the ```getMask()``` method.

In [ ]:
import numpy as nP

mask=w_seg.getMask()

print(mask.shape,np.unique(mask))

from PIL import Image as pImage
pImage.fromarray(mask)

The current category number when creating annotations is set by the ```targetClass``` attribute.

In [ ]:
w.targetClass=4

It is possible to modify the clsMap attribute which indicates the color of each class in the segmentation mask and if the class is in 'instance' or 'semantic' mode.

For each class number we associate a dictionary of 4 values, the first 3 encode the RGB color, the last indicates if we are in 'instance' (1) or 'semantic' (0) mode

In [ ]:
w.clsMap={0:{0,0,0,0},1:{255,0,0,0},2:{255,255,0,0},3:{0,0,255,0},4:{0,255,255,1},}
